# STL을 적용한 데이터로 LSTM 훈련

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import os

import pickle
import pandas as pd
import numpy as np
from tqdm import trange
import matplotlib.pyplot as plt
import matplotlib
import matplotlib.font_manager as font_manager
import joblib

In [ ]:
# 모델을 구축하기 위한 keras 관련 함수
from tensorflow.keras.models import *
from tensorflow.keras.layers import Lambda, RepeatVector
from tensorflow.keras.layers import Input, multiply
from tensorflow.keras.layers import Dense, LSTM, Dropout, Flatten
from tensorflow.keras import regularizers
import keras.backend as K
from keras.callbacks import EarlyStopping

## LSTM모델에 입력할 형태로 데이터 변환(window_size=24)

In [ ]:
def data_transform(name,time_steps):
    data=pd.read_csv("../Merged_Data/{:s}.csv".format(name))
    
    if name.split("x")[-1]=='(Seasonal_Trend)':
        
        data=data.drop("time",axis=1)
        sequences=[]
        for i in range(len(data) - time_steps + 1):
            sequence = data[i:i+time_steps]
            sequences.append(sequence)
        return np.array(sequences)
    
    return np.array(data[23:])

In [ ]:
train_x=data_transform('train_x(Seasonal_Trend)',24)
train_y=data_transform('train_y',24)
test_x=data_transform('test_x(Seasonal_Trend)',24)
test_y=data_transform('test_y',24)
valid_x=data_transform('valid_x(Seasonal_Trend)',24)
valid_y=data_transform('valid_y',24)

In [ ]:
print(train_x.shape)
print(train_y.shape)
print(valid_x.shape)

## LSTM모델 정의

In [ ]:
def lstm_model(batch_size, time_step, feature_num):
        """LSTM 모델 정의"""
        inputs = Input(shape=(time_step, feature_num))
        x = LSTM(6, activation='tanh',
        stateful=False,
        return_sequences=True,
        kernel_initializer='he_normal')(inputs)
        x = Dropout(0.2)(x)
        x = Flatten()(x)
        x = Dense(10, activation='linear', kernel_regularizer=regularizers.l2(0.01),
        activity_regularizer=regularizers.l1(0.))(x)
        output = Dense(1, activation='linear', kernel_initializer='he_normal')(x)

        model = Model(inputs=[inputs], outputs=output)
        
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, restore_best_weights=True)

        return model,early_stopping

## 하이퍼 파라미터 정의

In [ ]:
batch_size = 4
time_steps = 24
feature_num = 5

In [ ]:
model,early_stopping=lstm_model(batch_size, time_steps, feature_num)
model.compile(loss='mean_squared_error', optimizer='adam')

## 모델 훈련(epoch=100)

In [ ]:
history = model.fit(train_x, train_y,
                    validation_data=(valid_x, valid_y),
                    batch_size=batch_size, epochs=100)

## Loss를 시각화하여 학습 횟수의 적합성 확인

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']

# train loss와 validation loss의 변화를 matplotlib를 사용해 시각화함
plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.legend()

## 모델 저장, 성능 확인

In [ ]:
model.save('./Saved_Model/LSTM(epoch=100).pt')

In [ ]:
train_predict = model.predict(train_x, batch_size)
valid_predict = model.predict(valid_x, batch_size)
test_predict = model.predict(test_x, batch_size)

## sclaer를 불러와서 inverse transform 적용

In [ ]:
import joblib
scaler=joblib.load('../Merged_Data/Scaler/Y_pm10.pkl')

inv_train_y = scaler.inverse_transform(train_y)
inv_train_predict = scaler.inverse_transform(train_predict)
inv_valid_y = scaler.inverse_transform(valid_y)
inv_valid_predict = scaler.inverse_transform(valid_predict)
inv_test_y = scaler.inverse_transform(test_y)
inv_test_predict = scaler.inverse_transform(test_predict)

## 성능 평가 : SMAPE, RMSE

In [ ]:
def Symmetric_mean_absolute_percentage_error(actual, predicted):
    
    total=len(actual)
    numerator=np.abs(actual-predicted)
    denominator=np.abs(actual)+np.abs(predicted)
    SMAPE=(100/total)*np.sum(numerator/denominator)
    
    return SMAPE

from sklearn.metrics import mean_squared_error 

In [ ]:
train_smape = Symmetric_mean_absolute_percentage_error(inv_train_y, inv_train_predict)
valid_smape = Symmetric_mean_absolute_percentage_error(inv_valid_y, inv_valid_predict)
test_smape = Symmetric_mean_absolute_percentage_error(inv_test_y, inv_test_predict)

train_rmse = mean_squared_error (inv_train_y, inv_train_predict)
train_rmse=np.sqrt(train_rmse)
valid_rmse = mean_squared_error (inv_valid_y, inv_valid_predict)
valid_rmse=np.sqrt(valid_rmse)
test_rmse = mean_squared_error (inv_test_y, inv_test_predict)
test_rmse=np.sqrt(test_rmse)

In [ ]:
print("Train SMAPE:", train_smape)
print("Valid SMAPE:", test_smape)
print("Test SMAPE:", valid_smape)

print("Train RMSE:", train_rmse)
print("Valid RMSE:", test_rmse)
print("Test RMSE:", valid_rmse)

## 결과 시각화

In [ ]:
def vizualization_self(train_term,name,test_y,pred_y):
    plt.figure(figsize=(32, 16))
    
    plt.plot(np.arange(train_term), test_y[:train_term], color='red', ls='-', lw=3, label='Raw Data')
    plt.plot(np.arange(train_term), pred_y[:train_term], color='blue', ls='--', lw=3, label='Raw Data')
    
    plt.xlabel('[Time]', fontsize=25, fontweight='bold')
    plt.ylabel('[PM10]', fontsize=25, fontweight='bold')
    plt.title('Prediction Visualization({:s})'.format(name),fontsize=30, weight='bold')
    plt.xticks( fontsize=15, fontweight='bold')
    plt.yticks( fontsize=15, fontweight='bold')
    plt.grid(True)
    plt.legend()
    return

In [ ]:
vizualization_self(len(inv_test_y),'LSTM, epoch=100',inv_test_y,inv_test_predict)

In [ ]:
vizualization_self(200,'LSTM, epoch=100',inv_test_y,inv_test_predict)